# Case Study: How Does a Bike-Share Navigate Speedy Success?

## Case Study Background

This case study is about a bike-share company Cyclystic. Cyclistic has three pricing plan: single-ride passes. full-day passes, and annual membership. Single-ride or full-day passes are referred to as casual riders. The goal of Cyclistic marketing team is to maximizing the number of annual memberships by converting casual riders into annual members. 

The key question of this case study is: How do annual members and casual riders use Cyclistic bikes differently.

## Data Source

The datasets used is the divvy-tripdata from Motivate Internation Inc. (as Cyclysitc is a frictional company) from January 2021 to December 2021.

There are 12 different csv files with data of different months of 2021.

## Importing the data

In [1]:
# Initialize
import pandas as pd
import numpy as np
import geopy.distance
import os
pwd = os.getcwd()

# Importing Data to a list of DataFrame
data=[]
for i in range(12):
    filepath = pwd + "\DataSource/" + str(202101+i) + "-divvy-tripdata.csv"
    data += [pd.read_csv(filepath)]

# Check if the 12 DataFrame have same columns
for i in range(11):
    if  not data[11].columns.equals(data[i].columns):
        print("columns not identical")

# Checking if the 12 csv have the same columuns
for i in range(11):
    if  not data[11].columns.equals(data[i].columns):
        print("columns not identical")

# Merging the data
MergedData = pd.concat(data)

# Reset the index
MergedData.reset_index(drop=True)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,E19E6F1B8D4C42ED,electric_bike,2021-01-23 16:14:19,2021-01-23 16:24:44,California Ave & Cortez St,17660,NaN,NaN,41.900341,-87.696743,41.890000,-87.720000,member
1,DC88F20C2C55F27F,electric_bike,2021-01-27 18:43:08,2021-01-27 18:47:12,California Ave & Cortez St,17660,NaN,NaN,41.900333,-87.696707,41.900000,-87.690000,member
2,EC45C94683FE3F27,electric_bike,2021-01-21 22:35:54,2021-01-21 22:37:14,California Ave & Cortez St,17660,NaN,NaN,41.900313,-87.696643,41.900000,-87.700000,member
3,4FA453A75AE377DB,electric_bike,2021-01-07 13:31:13,2021-01-07 13:42:55,California Ave & Cortez St,17660,NaN,NaN,41.900399,-87.696662,41.920000,-87.690000,member
4,BE5E8EB4E7263A0B,electric_bike,2021-01-23 02:24:02,2021-01-23 02:24:45,California Ave & Cortez St,17660,NaN,NaN,41.900326,-87.696697,41.900000,-87.700000,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5595058,847431F3D5353AB7,electric_bike,2021-12-12 13:36:55,2021-12-12 13:56:08,Canal St & Madison St,13341,NaN,NaN,41.882289,-87.639752,41.890000,-87.610000,casual
5595059,CF407BBC3B9FAD63,electric_bike,2021-12-06 19:37:50,2021-12-06 19:44:51,Canal St & Madison St,13341,Kingsbury St & Kinzie St,KA1503000043,41.882123,-87.640053,41.889106,-87.638862,member
5595060,60BB69EBF5440E92,electric_bike,2021-12-02 08:57:04,2021-12-02 09:05:21,Canal St & Madison St,13341,Dearborn St & Monroe St,TA1305000006,41.881956,-87.639955,41.880254,-87.629603,member
5595061,C414F654A28635B8,electric_bike,2021-12-13 09:00:26,2021-12-13 09:14:39,Lawndale Ave & 16th St,362.0,NaN,NaN,41.860000,-87.720000,41.850000,-87.710000,member


### Quick glance of the Data

In [2]:
MergedData.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')

## Data preprocesing 

#### Converting the data type of "started_at" and "ended_at" to datetime

In [3]:
MergedData["started_at"]=pd.to_datetime(MergedData["started_at"])
MergedData["ended_at"]=pd.to_datetime(MergedData["ended_at"])

#### Adding a new column "TravelTime" representing the time used in each trip by minutes 

In [4]:
#Covert to seconds then divide 60 with round function to aviod rounding up cause by .astype()
MergedData["TravelTime"]=((MergedData["ended_at"]-MergedData["started_at"]).astype("timedelta64[s]")/60).round(0)

#### Adding Columns "DayOfWeek" representing the day of week of the start date and "DayOfWeekint" representing the day of week by 0 as Monday and 6 as Sunday

In [5]:
MergedData["DayOfWeekint"]=(MergedData["ended_at"].dt.dayofweek)
MergedData["DayOfWeek"]=MergedData["DayOfWeekint"].apply(lambda x:["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"][x])

#### Adding a new column "Traveldistance" representing the distance between start and end station by meters

In [6]:
def dist(slat,slng,elat,elng):
    for x in [slat,slng,elat,elng]:
        if np.isnan(x):
            return np.nan
    return geopy.distance.geodesic((slat,slng),(elat,elng)).m

MergedData["TravelDistance"]=MergedData.apply(lambda row: dist(row.start_lat,row.start_lng,row.end_lat,row.end_lng), axis=1)


### Removing unwanted Data

### Checking for duplicates

In [8]:
MergedData[MergedData.duplicated()]

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,TravelTime,DayOfWeekint,DayOfWeek,TravelDistance


#### Checking Data Errors with started time after ended time.

MergedData[MergedData["TravelTime"]<0]

As there are only a ignorable amount of rows of error we will omit the wrong data.

In [9]:
MergedData=MergedData.drop(index=MergedData[MergedData["TravelTime"]<0].index)

### Checking trips with same start and end station and time less than half a minute.
These trips are mostlikey be trips that is made by mistake. We will first analysis the number of these trips and remove these trip as they will be irrelevant for the analysis.

In [10]:
NullTrips=MergedData[(MergedData["TravelDistance"]==0) & (MergedData["TravelTime"]==0)]
print("Total number of trips :",MergedData["ride_id"].size)
print("Total number of trips made by mistake:",NullTrips["ride_id"].size,"  Percentage: {:.3%}".format(NullTrips["ride_id"].size/MergedData["ride_id"].size))
print("Number of trips made by mistake:")
df=NullTrips.groupby("member_casual")["ride_id"].size().reset_index()
df=df.rename(columns={"ride_id":"Number_of_trips"})
df["Percentage"]=(df["Number_of_trips"]/df["Number_of_trips"].sum()*100).round(2)
df

Total number of trips : 5594312
Total number of trips made by mistake: 42838   Percentage: 0.766%
Number of trips made by mistake:


,member_casual,Number_of_trips,Percentage
0,casual,17165,40.07
1,member,25673,59.93


#### Removing trips that are made by mistake

In [11]:
MergedData=MergedData.drop(index=MergedData[(MergedData["TravelDistance"]==0) & (MergedData["TravelTime"]==0)].index)

In [12]:
## Export data for data viz
MergedData.to_csv("ProcessedData.csv")

#### Removing Columns start_lat, start_lng, end_lat, and end_lng as we will not be using these columns in our analysis.

In [13]:
MergedData=MergedData.drop(columns=["start_station_id","end_station_id","start_lat","start_lng","end_lat","end_lng"])

In [14]:
MergedData.reset_index(drop=True)

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual,TravelTime,DayOfWeekint,DayOfWeek,TravelDistance
0,E19E6F1B8D4C42ED,electric_bike,2021-01-23 16:14:19,2021-01-23 16:24:44,California Ave & Cortez St,NaN,member,10.0,5,Saturday,2245.917159
1,DC88F20C2C55F27F,electric_bike,2021-01-27 18:43:08,2021-01-27 18:47:12,California Ave & Cortez St,NaN,member,4.0,2,Wednesday,557.774217
2,EC45C94683FE3F27,electric_bike,2021-01-21 22:35:54,2021-01-21 22:37:14,California Ave & Cortez St,NaN,member,1.0,3,Thursday,280.748233
3,4FA453A75AE377DB,electric_bike,2021-01-07 13:31:13,2021-01-07 13:42:55,California Ave & Cortez St,NaN,member,12.0,3,Thursday,2246.220360
4,BE5E8EB4E7263A0B,electric_bike,2021-01-23 02:24:02,2021-01-23 02:24:45,California Ave & Cortez St,NaN,casual,1.0,5,Saturday,276.456009
...,...,...,...,...,...,...,...,...,...,...,...
5268418,847431F3D5353AB7,electric_bike,2021-12-12 13:36:55,2021-12-12 13:56:08,Canal St & Madison St,NaN,casual,19.0,6,Sunday,2613.695258
5268419,CF407BBC3B9FAD63,electric_bike,2021-12-06 19:37:50,2021-12-06 19:44:51,Canal St & Madison St,Kingsbury St & Kinzie St,member,7.0,0,Monday,781.901574
5268420,60BB69EBF5440E92,electric_bike,2021-12-02 08:57:04,2021-12-02 09:05:21,Canal St & Madison St,Dearborn St & Monroe St,member,8.0,3,Thursday,879.844676
5268421,C414F654A28635B8,electric_bike,2021-12-13 09:00:26,2021-12-13 09:14:39,Lawndale Ave & 16th St,NaN,member,14.0,0,Monday,1386.796868


## Analysis

#### Number of Trips by membership

In [15]:
df=pd.DataFrame(data={"Count":MergedData.groupby("member_casual")["ride_id"].count()})
df["Percentage"]=df["Count"]/df["Count"].sum()*100
print("Total Number of trips:")
df

Total Number of trips:


,Count,Percentage
member_casual,,
casual,2388025,45.327131
member,2880398,54.672869


#### Number of Trips that start date and end date are different

In [16]:
MergedData[MergedData["started_at"].dt.date!=MergedData["ended_at"].dt.date].groupby("member_casual")["ride_id"].count()

member_casual
casual    31114
member     9955
Name: ride_id, dtype: int64

#### Number of Trips that start and end station are same

In [17]:
MergedData[MergedData["TravelDistance"]==0].groupby("member_casual")["ride_id"].count()

member_casual
casual    202803
member    112760
Name: ride_id, dtype: int64

#### The Mean, Mode, Meadian, Min and Max of travel time by membership

In [18]:
# Seperate the data into two dataframe for member and causal.
md=MergedData[MergedData["member_casual"]=="member"]
cd=MergedData[MergedData["member_casual"]=="casual"]

membertimemean=md["TravelTime"].mean()
casualtimemean=cd["TravelTime"].mean()
print("Travel time by membership:")
pd.DataFrame(data={"Membership":["Member","Casual"],
"Mean":[md["TravelTime"].mean(),cd["TravelTime"].mean()],
"Mode":[int(md["TravelTime"].mode()),int(cd["TravelTime"].mode())],
"Median":[md["TravelTime"].median(),cd["TravelTime"].median()],
"Min":[md["TravelTime"].min(),cd["TravelTime"].min()],
"Max":[md["TravelTime"].max(),cd["TravelTime"].max()]})

Travel time by membership:


,Membership,Mean,Mode,Median,Min,Max
0,Member,13.74940,6,10.0,0.0,1560.0
1,Casual,32.26108,8,16.0,0.0,55944.0


#### The statistics of Members on Travel time by day of week

In [19]:
Mtimestat=md.groupby("DayOfWeek").agg({"TravelTime":["count","median","mean","max"]}).sort_values(by="DayOfWeek",key=lambda x:x.map(["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"].index))
Mtimestat["percentage"]=(Mtimestat[("TravelTime","count")]/Mtimestat[("TravelTime","count")].sum()*100).round(2)
print("Travel time Statistics of Members:")
Mtimestat

Travel time Statistics of Members:


TravelTime                           percentage
               count median       mean     max           
DayOfWeek                                                
Monday        391072    9.0  13.385241  1560.0      13.58
Tuesday       437369    9.0  12.988769  1500.0      15.18
Wednesday     448574    9.0  12.931791  1500.0      15.57
Thursday      424066    9.0  12.833064  1500.0      14.72
Friday        418363   10.0  13.359465  1500.0      14.52
Saturday      406165   11.0  15.343343  1500.0      14.10
Sunday        354789   11.0  15.852521  1560.0      12.32

In [20]:
Ctimestat=cd.groupby("DayOfWeek").agg({"TravelTime":["count","median","mean","max"]}).sort_values(by="DayOfWeek",key=lambda x:x.map(["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"].index))
Ctimestat["percentage"]=(Ctimestat[("TravelTime","count")]/Ctimestat[("TravelTime","count")].sum()*100).round(2)
print("Travel time Statistics of Casuals:")
Ctimestat

Travel time Statistics of Casuals:


TravelTime                            percentage
               count median       mean      max           
DayOfWeek                                                 
Monday        271872   16.0  36.188449  52701.0      11.38
Tuesday       259194   14.0  32.798055  55944.0      10.85
Wednesday     263781   14.0  30.331267  49107.0      11.05
Thursday      269393   14.0  29.172406  47011.0      11.28
Friday        340915   15.0  29.064210  41646.0      14.28
Saturday      524026   18.0  31.430231  40725.0      21.94
Sunday        458844   19.0  35.877647  40705.0      19.21

#### Number of Trips by Months

In [21]:
pd.DataFrame(data={("Member","Count"):md.groupby(md["started_at"].dt.month).size(),
("Member","Percentage"):md.groupby(md["started_at"].dt.month).size()/md.index.size*100,
("Casual","Count"):cd.groupby(cd["started_at"].dt.month).size(),
("Casual","Percentage"):cd.groupby(cd["started_at"].dt.month).size()/cd.index.size*100})

Member             Casual           
             Count Percentage   Count Percentage
started_at                                      
1            70972   2.463965   16298   0.682489
2            34421   1.195008    8889   0.372232
3           133427   4.632242   77726   3.254824
4           187115   6.496151  127404   5.335120
5           258080   8.959873  242095  10.137875
6           339492  11.786288  351198  14.706630
7           359918  12.495426  420693  17.616775
8           372113  12.918805  391115  16.378179
9           370869  12.875616  345216  14.456130
10          353545  12.274172  242728  10.164383
11          236031   8.194388  100072   4.190576
12          164415   5.708065   64591   2.704787

In [22]:
MtimestatM=md.groupby(md["started_at"].dt.month).agg({"TravelTime":["count","median","mean","max"]})
MtimestatM["percentage"]=(MtimestatM[("TravelTime","count")]/MtimestatM[("TravelTime","count")].sum()*100).round(2)
print("Travel time Statistics of Members by Months:")
MtimestatM

Travel time Statistics of Members by Months:


TravelTime                           percentage
                count median       mean     max           
started_at                                                
1               70972    9.0  12.859170  1500.0       2.46
2               34421   10.0  18.357108  1500.0       1.20
3              133427   10.0  14.080988  1560.0       4.63
4              187115   10.0  14.819389  1500.0       6.50
5              258080   11.0  14.778681  1500.0       8.96
6              339492   11.0  14.823257  1500.0      11.79
7              359918   11.0  14.386177  1500.0      12.50
8              372113   10.0  14.213470  1500.0      12.92
9              370869   10.0  13.849723  1500.0      12.88
10             353545    9.0  12.592519  1500.0      12.27
11             236031    8.0  11.396240  1500.0       8.19
12             164415    8.0  11.044479  1500.0       5.71

In [23]:
CtimestatM=cd.groupby(cd["started_at"].dt.month).agg({"TravelTime":["count","median","mean","max"]})
CtimestatM["percentage"]=(CtimestatM[("TravelTime","count")]/CtimestatM[("TravelTime","count")].sum()*100).round(2)
print("Travel time Statistics of Casuals by Months:")
CtimestatM

Travel time Statistics of Casuals by Months:


TravelTime                            percentage
                count median       mean      max           
started_at                                                 
1               16298   12.0  25.793533  19826.0       0.68
2                8889   16.0  51.360220  30130.0       0.37
3               77726   19.0  38.482901  31682.0       3.25
4              127404   18.0  38.259803  47777.0       5.34
5              242095   19.0  38.247324  53921.0      10.14
6              351198   18.0  37.598264  55944.0      14.71
7              420693   17.0  33.090436  49107.0      17.62
8              391115   16.0  29.033514  41629.0      16.38
9              345216   15.0  27.974665  32859.0      14.46
10             242728   14.0  28.982301  40705.0      10.16
11             100072   11.0  23.354075  34998.0       4.19
12              64591   11.0  23.660897  30401.0       2.70

#### Top 10 Start station

In [24]:
print("Top 10 Start station for Members")
md.groupby('start_station_name').size().sort_values(ascending=False)[0:10]


Top 10 Start station for Members


start_station_name
Clark St & Elm St            23278
Wells St & Concord Ln        22313
Kingsbury St & Kinzie St     22308
Wells St & Elm St            19936
Dearborn St & Erie St        18431
Wells St & Huron St          18172
St. Clair St & Erie St       17789
Broadway & Barry Ave         16748
Desplaines St & Kinzie St    15939
Clinton St & Madison St      15793
dtype: int64

In [25]:
print("Top 10 Start station for Casuals")
cd.groupby('start_station_name').size().sort_values(ascending=False)[0:10]

Top 10 Start station for Casuals


start_station_name
Streeter Dr & Grand Ave       63059
Millennium Park               31678
Michigan Ave & Oak St         28353
Shedd Aquarium                21963
Theater on the Lake           20148
Wells St & Concord Ln         18793
Lake Shore Dr & Monroe St     18472
Clark St & Lincoln Ave        16252
Wells St & Elm St             15890
Indiana Ave & Roosevelt Rd    15675
dtype: int64